In [1]:
#-------- Import Libraries --------#
import torch
import os
import sys
import random
import mlflow
import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sn
import matplotlib.pyplot as plt
from datetime import date
from sklearn.metrics import matthews_corrcoef
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc

In [2]:
#-------- Import Modules from project--------#
import encoding as enc
from model import Net, Net_th
import functions as func

In [3]:
#-------- Set Device --------#

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
else:
    print('No GPUs available. Using CPU instead.')
    device = torch.device('cpu')

No GPUs available. Using CPU instead.


In [4]:
#-------- Seeds --------#

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

torch.use_deterministic_algorithms(True)

In [5]:
#-------- Directories --------#

DATADIR = '/data/'
TRAINDIR = '../data/train'
VALIDATIONDIR = '../data/validation'
MATRICES = '/data/Matrices'

In [6]:
#-------- Unzip Train --------#

if len(os.listdir(TRAINDIR)) != 0:
        print("{} already unzipped.".format(TRAINDIR))
else:
    !unzip ../data/train.zip -d ../data/train

    
#-------- Unzip Validation --------#


if len(os.listdir(VALIDATIONDIR)) != 0:
        print("{} already unzipped.".format(VALIDATIONDIR))
else:
    !unzip ../data/validation.zip -d ../data/validation
    
print('Train directory:\n\n', '\n'.join(str(p) for p in os.listdir(TRAINDIR)), '\n\n')
print('Validation directory:\n\n','\n'.join(str(p) for p in os.listdir(VALIDATIONDIR)))

../data/train already unzipped.
../data/validation already unzipped.
Train directory:

 P1_input.npz
P1_labels.npz
P2_input.npz
P2_labels.npz
P3_input.npz
P3_labels.npz
P4_input.npz
P4_labels.npz
__MACOSX 


Validation directory:

 P4_input.npz
P4_labels.npz
__MACOSX


In [7]:
#-------- Import Dataset --------#             #TO UPDATE: NO REPEAT P4 AND CONSERVE PARTITION FOR CROSS VAL

data_list = []
target_list = []

import glob
for fp in glob.glob("../data/train/*input.npz"):
    data = np.load(fp)["arr_0"]
    targets = np.load(fp.replace("input", "labels"))["arr_0"]
    data_list.append(data)
    target_list.append(targets)
    
print(len(data_list))
print(len(target_list))

X_train = np.concatenate(data_list[ :-1])
y_train = np.concatenate(target_list[:-1])
nsamples, nx, ny = X_train.shape
print("Training set shape:", nsamples,nx,ny)

X_val = np.concatenate(data_list[-1: ])
y_val = np.concatenate(target_list[-1: ])
nsamples, nx, ny = X_val.shape
print("Val set shape:", nsamples,nx,ny)

p_neg = len(y_train[y_train == 1])/len(y_train)*100
print("Percent positive samples in train:", p_neg)

p_pos = len(y_val[y_val == 1])/len(y_val)*100
print("Percent positive samples in val:", p_pos)

# make the data set into one dataset that can go into dataloader
train_ds = []
for i in range(len(X_train)):
    train_ds.append([np.transpose(X_train[i]), y_train[i]])

val_ds = []
for i in range(len(X_val)):
    val_ds.append([np.transpose(X_val[i]), y_val[i]])

bat_size = 64
print("\nNOTE:\nSetting batch-size to", bat_size)
train_ldr = torch.utils.data.DataLoader(train_ds,batch_size=bat_size, shuffle=True)
val_ldr = torch.utils.data.DataLoader(val_ds,batch_size=bat_size, shuffle=True)

4
4
Training set shape: 4174 420 54
Val set shape: 1532 420 54
Percent positive samples in train: 24.96406324868232
Percent positive samples in val: 25.0

NOTE:
Setting batch-size to 64


In [8]:
 # Hyperparameters to fine-tune
embedding = "Baseline"
numHN=64
numFilter=100
dropOutRate=0.1

In [9]:
#fixed hyperparameters
features = list(range(54))  #to be redefined
residues = list(range(416))  #to be redefined
num_classes = 1
learning_rate = 0.001
n_features = len(features)
input_size = len(residues)
bat_size = 128
epochs = 100

In [10]:
#data load


#-------- Train --------#
X_train = np.concatenate(data_list[ :-1])
y_train = np.concatenate(target_list[:-1])
nsamples, nx, ny = X_train.shape
print("Training set shape:", nsamples,nx,ny)

X_valid = np.concatenate(data_list[-1: ])
y_valid = np.concatenate(target_list[-1: ])
nsamples, nx, ny = X_valid.shape
print("Validation set shape:", nsamples,nx,ny)

# Dataloader
train_ds = []
for i in range(len(X_train)):
    train_ds.append([np.transpose(X_train[i][:,features]), y_train[i]])
val_ds = []
for i in range(len(X_valid)):
    val_ds.append([np.transpose(X_valid[i][:,features]), y_valid[i]])
train_ldr = torch.utils.data.DataLoader(train_ds,batch_size=bat_size, shuffle=True)
val_ldr = torch.utils.data.DataLoader(val_ds,batch_size=bat_size, shuffle=True)

Training set shape: 4174 420 54
Validation set shape: 1532 420 54


In [11]:
# Initialize network
net = Net_project(num_classes=num_classes, 
             n_features=n_features, 
             numHN=numHN, 
             numFilter=numFilter,
             dropOutRate=dropOutRate).to(device)

# Optimizer
optimizer = optim.Adam(net.parameters(), lr=learning_rate,
                        weight_decay=0.0005,
                        amsgrad=True,
                        )

#train
train_acc, train_losses, train_auc, valid_acc, valid_losses, valid_auc, val_preds, val_targs = func.train_th(net, optimizer, train_ldr, val_ldr, [], [], epochs, criterion)

#-------- Performance --------#
epoch = np.arange(1,len(train_losses)+1)
plt.figure()
plt.plot(epoch, train_losses, 'r', epoch, valid_losses, 'b')
plt.legend(['Train Loss','Validation Loss'])
plt.xlabel('Epoch'), plt.ylabel('Loss')

epoch = np.arange(1,len(train_auc)+1)
plt.figure()
plt.plot(epoch, train_auc, 'r', epoch, valid_auc, 'b')
plt.legend(['Train AUC','Validation AUC'])
plt.xlabel('Epoch'), plt.ylabel('AUC')

epoch = np.arange(1,len(train_acc)+1)
plt.figure()
plt.plot(epoch, train_acc, 'r', epoch, valid_acc, 'b')
plt.legend(['Train Accuracy','Validation Accuracy'])
plt.xlabel('Epoch'), plt.ylabel('Acc')
plt.show()

#save results
results = pd.DataFrame(list(zip( (int(x) for x in val_targs), (int(x) for x in val_preds))),columns =['target', 'pred'])
print(results)

#metrics
AUC = roc_auc_score(results['target'], results['pred'])
MCC = matthews_corrcoef(results['target'], results['pred'])
print("AUC: ", AUC)
print("MCC: ", MCC)

confusion_matrix = pd.crosstab(results['target'], results['pred'], rownames=['Actual'], colnames=['Predicted'])
sn.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g')
plt.show()

print( len([i for i, (a, b) in enumerate(zip(results['pred'], results['target'])) if a != b]))

AttributeError: module 'functions' has no attribute 'train_th'

In [ ]:
#storing values
with mlflow.start:run():
    mlflow.log_param('embedding', embedding)
    mlflow.log_metric('AUC', AUC)
    mlflow.log_metric('MCC', MCC)
    